In [1]:
import torch

In [2]:
cos_cached = None
sin_cached = None

In [3]:
B = 2 # batch_sz
seq_len = 32
embd_sz = 512 
nh = 4 # n_heads
hs = int(embd_sz / nh) # head_sz -> 512 / n_heads
hs

128

In [4]:
q = torch.randn((B, nh, seq_len, hs)) # B, nh, T, hs
q.shape

torch.Size([2, 4, 32, 128])

In [5]:
torch.arange(0, hs, 2)

tensor([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,  26,
         28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,  52,  54,
         56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,  78,  80,  82,
         84,  86,  88,  90,  92,  94,  96,  98, 100, 102, 104, 106, 108, 110,
        112, 114, 116, 118, 120, 122, 124, 126])

In [6]:
2*torch.arange(0, hs//2)

tensor([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,  26,
         28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,  52,  54,
         56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,  78,  80,  82,
         84,  86,  88,  90,  92,  94,  96,  98, 100, 102, 104, 106, 108, 110,
        112, 114, 116, 118, 120, 122, 124, 126])

In [7]:
theta = 1. / (10_000 ** (torch.arange(0, hs, 2) / hs).float())
theta.shape

torch.Size([64])

In [8]:
seq_idx = torch.arange(seq_len).float()
seq_idx.shape

torch.Size([32])

In [9]:
idx_theta = torch.einsum('n,d->nd', seq_idx, theta)
idx_theta.shape

torch.Size([32, 64])

In [10]:
idx_theta2 = torch.repeat_interleave(idx_theta, 2, dim=-1)
idx_theta2.shape

torch.Size([32, 128])

In [11]:
idx_theta2

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 1.0000e+00, 8.6596e-01,  ..., 1.3335e-04, 1.1548e-04,
         1.1548e-04],
        [2.0000e+00, 2.0000e+00, 1.7319e+00,  ..., 2.6670e-04, 2.3096e-04,
         2.3096e-04],
        ...,
        [2.9000e+01, 2.9000e+01, 2.5113e+01,  ..., 3.8672e-03, 3.3489e-03,
         3.3489e-03],
        [3.0000e+01, 3.0000e+01, 2.5979e+01,  ..., 4.0006e-03, 3.4643e-03,
         3.4643e-03],
        [3.1000e+01, 3.1000e+01, 2.6845e+01,  ..., 4.1339e-03, 3.5798e-03,
         3.5798e-03]])

In [12]:
idx_theta2.cos()[None, None, :, :].shape

torch.Size([1, 1, 32, 128])

In [13]:
cos_cached = idx_theta2.cos()[None, None, :, :]
sin_cached = idx_theta2.sin()[None, None, :, :]

cos_cached.shape, sin_cached.shape

(torch.Size([1, 1, 32, 128]), torch.Size([1, 1, 32, 128]))

In [14]:
d_2 = hs//2
d_2

64

In [15]:
q.shape

torch.Size([2, 4, 32, 128])

In [16]:
(q * cos_cached).shape

torch.Size([2, 4, 32, 128])

In [17]:
torch.arange(0, q.shape[-1], 2)

tensor([  0,   2,   4,   6,   8,  10,  12,  14,  16,  18,  20,  22,  24,  26,
         28,  30,  32,  34,  36,  38,  40,  42,  44,  46,  48,  50,  52,  54,
         56,  58,  60,  62,  64,  66,  68,  70,  72,  74,  76,  78,  80,  82,
         84,  86,  88,  90,  92,  94,  96,  98, 100, 102, 104, 106, 108, 110,
        112, 114, 116, 118, 120, 122, 124, 126])

In [18]:
t = torch.arange(1, 9)
print(t)
t = t.view(-1, 2).flip(dims=[1])
t[:, 0] = -t[:, 0]
t = t.view(-1)
t

tensor([1, 2, 3, 4, 5, 6, 7, 8])


tensor([-2,  1, -4,  3, -6,  5, -8,  7])

In [19]:
t = torch.arange(1, 9).unsqueeze(0)
print(t)
t[..., 1::2] = -t[..., 1::2]
t = t.view(-1, 2).flip(dims=[1]).view(1, -1)
t

tensor([[1, 2, 3, 4, 5, 6, 7, 8]])


tensor([[-2,  1, -4,  3, -6,  5, -8,  7]])

In [20]:
q_shape = q.shape
q_c = q.clone()
q_c[..., 1::2] = -q_c[..., 1::2]
q_c = q_c.view(-1, 2).flip(dims=[1]).view(*q_shape)
q_c.shape

torch.Size([2, 4, 32, 128])

In [21]:
(q_c * sin_cached).shape

torch.Size([2, 4, 32, 128])

In [22]:
rope = (q * cos_cached) + (q_c * sin_cached)
rope.shape

torch.Size([2, 4, 32, 128])

In [23]:
from rotary_embedding_torch import RotaryEmbedding

rotary_emb = RotaryEmbedding(dim=hs)

In [24]:
rot_emb = rotary_emb.rotate_queries_or_keys(q)
rot_emb.shape

torch.Size([2, 4, 32, 128])

In [25]:
torch.allclose(rope, rot_emb)

True

In [26]:
def compute_rope(q):
    global cos_cached
    global sin_cached
    q2 = q.clone()
    q2[..., 1::2] = -q2[..., 1::2]
    q2 = q2.view(-1, 2).flip(dims=[1]).view(*q.shape)
    return (q * cos_cached) + (q2 * sin_cached)


In [27]:
torch.allclose(rot_emb, compute_rope(q))

True

In [28]:
from model import RotaryPositionalEmbedding

In [29]:
r = RotaryPositionalEmbedding(hs, seq_len)

In [35]:
torch.allclose(r(q), rotary_emb.rotate_queries_or_keys(q))

True

In [33]:
%timeit RotaryPositionalEmbedding(hs, seq_len)

192 µs ± 12.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [36]:
%timeit r(q)

208 µs ± 3.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
